# Vowpal Wabbit в NLP
Автоматическая обработка текстов - 2017, семинар 4.

В этом семинаре мы познакомимся с библиотекой Vowpal Wabbit и решим с его помощью задачу многоклассовой классификации на больших данных. 
Данные скачайте [здесь](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data) или запустите следующие две ячейки.

! wget https://www.dropbox.com/s/r0q0p0uprhcp8bb/train-sample.zip
! unzip train-sample.zip

! wget https://www.dropbox.com/s/50vw2gsglc91f6o/train.zip
! unzip train.zip

Чтобы на семинаре не тратить время на обработку и обучение моделей на всех данных, предлагается использовать только небольшую подвыборку (`train-sample.csv`). Но сдавать ноутбук все равно необходимо с результатами на **всех** данных.

In [1]:
import csv

INPUT_DATA = 'train.csv'
#INPUT_DATA = 'train-sample.csv'

reader = csv.DictReader(open(INPUT_DATA))
dict(next(reader))

{'BodyMarkdown': "I'm new to C#, and I want to use a trackbar for the forms opacity\r\nThis is my code\r\n\r\n    decimal trans = trackBar1.Value / 5000\r\n    this.Opacity = trans\r\n\r\nWhen I try to build it, I get this error\r\n\r\n**Cannot implicitly convert type 'decimal' to 'double**\r\n\r\nI tried making trans a double, but then the control doesn't work. This code worked fine for me in VB.NET. Any suggestions?",
 'OpenStatus': 'open',
 'OwnerCreationDate': '07/31/2008 21:33:24',
 'OwnerUndeletedAnswerCountAtPostTime': '0',
 'OwnerUserId': '8',
 'PostClosedDate': '',
 'PostCreationDate': '07/31/2008 21:42:52',
 'PostId': '4',
 'ReputationAtPostCreation': '1',
 'Tag1': 'c#',
 'Tag2': '',
 'Tag3': '',
 'Tag4': '',
 'Tag5': '',
 'Title': 'Decimal vs Double?'}

Каждый объект выборки соответствует некоторому посту на Stack Overflow. Требуется построить модель, определяющую статус поста. Подробнее про задачу и формат данных можно прочитать на [странице соревнования](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow).

Перед обучением модели из Vowpal Wabbit данные следует сохранить в специальный формат: <br>
`label |namespace1 feature1:value1 feature2 feature3:value3 |namespace2 ...` <br>
Записи `feature` и `feature:1.0` эквивалентны. Выделение признаков в смысловые подгруппы (namespaces) позволяет создавать взаимодействия между ними. Подробнее про формат входных данных можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format).

Ниже реализована функция, которая извлекает признаки с помощью подаваемого на вход экстрактора, разбивает данные на трейн и тест и записывает их на диск.

In [2]:
STATUSES = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']
STATUS_DICT = {status: i+1 for i, status in enumerate(STATUSES)}

def data2vw(features_extractor, train_output='train', test_output='test', ytest_output='ytest'):
    reader = csv.DictReader(open(INPUT_DATA))
    writer_train = open(train_output, 'w')
    writer_test = open(test_output, 'w')
    writer_ytest = open(ytest_output, 'w')
    
    for row in reader:
        label = STATUS_DICT[row['OpenStatus']]
        features = features_extractor(row)
        output_line = '%s %s\n' % (label, features)
        if int(row['PostId']) % 2 == 0:
            writer_train.write(output_line)
        else:
            writer_test.write(output_line)
            writer_ytest.write('%s\n' % label)
            
    writer_train.close()
    writer_test.close()
    writer_ytest.close()

Начнем с простейшей модели. В качестве признаков возьмите заголовки и очистите их: приведите символы к нижнему регистру, удалите пунктуацию. Также приветствуется использование стеммеров/лемматизаторов, однако учтите, что они могут сильно замедлить скорость обработки.

In [3]:
import re

def extract_title(row):
    title = row['Title']
    # YOUR CODE HERE
    return ''.join([character for character in title.lower() if (character.isalpha() or character == ' ')])

In [4]:
data2vw(lambda row: '| %s' % extract_title(row))

In [5]:
! head -n 5 train

4 | decimal vs double
4 | percentage width child in absolutely positioned parent doesnt work in ie
4 | tools for porting j code to c
4 | throw error in mysql trigger
4 | whats the difference between mathfloor and mathtruncate


Обучим `vw` модель. Параметр `-d` отвечает за путь к обучающей выборке, `-f` – за путь к модели, `--oaa` – за режим мультиклассовой классификации `one-against-all`. Подробное описание всех параметров можно найти [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) или вызвав `vw --help`.

In [6]:
! vw -d train --loss_function logistic --oaa 5 -f model

final_regressor = model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4        1        4
0.500000 0.000000            2            2.0        4        4       12
0.250000 0.000000            4            4.0        4        4        6
0.125000 0.000000            8            8.0        4        4        7
0.062500 0.000000           16           16.0        4        4        4
0.062500 0.062500           32           32.0        4        4        9
0.046875 0.031250           64           64.0        4        4        5
0.062500 0.078125          128          128.0        4        4       11
0.089844 0.117188          256          256.0        4        4        8
0.070312 0.050781          512          512.0   

Применим модель к тестовой выборке и сохраним предсказания.

In [7]:
! vw -i model -t test -r pred

only testing
raw predictions = pred
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        4        4        9
0.000000 0.000000            2            2.0        4        4        7
0.000000 0.000000            4            4.0        4        4        8
0.125000 0.250000            8            8.0        4        4        6
0.125000 0.125000           16           16.0        4        4        7
0.156250 0.187500           32           32.0        4        4       10
0.093750 0.031250           64           64.0        4        4        8
0.117188 0.140625          128          128.0        4        4        8
0.125000 0.132812          256          256.0        4        4       15
0.109375 0.093750          512       

In [8]:
! head -n 3 pred

1:-4.35195 2:-5.91848 3:-4.50574 4:2.55012 5:-6.68008
1:-7.03339 2:-7.67415 3:-6.85027 4:4.34536 5:-6.29331
1:-4.26178 2:-5.55279 3:-5.30194 4:2.89059 5:-7.50815


Реализуйте функцию, которая вычисляет `logloss` и `accuracy`, не загружая вектора в память. Используйте `softmax`, чтобы получить вероятности.

In [9]:
def get_scores(ytest_input='ytest', pred_input='pred'):
    import numpy as np
    n, error, loss = 0, 0, 0
    reader_ytest = open(ytest_input, 'r')
    reader_pred = open(pred_input, 'r')
    
    for label, pred in zip(reader_ytest, reader_pred):
        # YOUR CODE HERE
        scores = np.array([
            float(item[item.find(':') + 1:])
            for item in pred.split(' ')
        ])
        label = int(label)
        probabilities = np.exp(scores) / np.sum(np.exp(scores))
        loss += np.log(probabilities[label - 1])
        if ((np.argmax(probabilities) + 1) != label):
            error += 1
        n += 1

    reader_ytest.close()
    reader_pred.close()
    return - loss / n, 1 - float(error) / n

In [10]:
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14684
accuracy = 0.97793


На оригинальных данных `logloss` должен быть меньше `0.20`, `accuracy` больше `0.95`. Если это не так, то скорее всего у вас ошибка.

Теперь попробуем улучшить модель, добавив новые признаки, порождаемые словами. В `vowpal wabbit` есть возможность делать это прямо на лету. Воспользуйтесь параметрами `affix`, `ngram`, `skips`.

Далее везде при подборе параметров ориентируйтесь на улучшение `logloss`. Используйте `--quiet` или `-P`, чтобы избавиться от длинных выводов при обучении и применении моделей.

In [11]:
# YOUR CODE HERE
# Добавляем n-граммы разных длин
! vw -d train --loss_function logistic --oaa 5 -f model --ngram 2 --ngram 3 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.18119
accuracy = 0.97577


In [12]:
# Добавляем n-граммы и n-skips разных длин
! vw -d train --loss_function logistic --oaa 5 -f model --skips 3 --ngram 4 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.16338
accuracy = 0.97785


In [13]:
# Добавляем суффиксы
! vw -d train --loss_function logistic --oaa 5 -f model --affix +2 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13970
accuracy = 0.97818


Часто качество `vw` модели получается учушить увеличением числа проходов по обучающей выборке (параметр `--passes`) и увеличением числа бит хэш-функции для уменьшения числа коллизий признаков (параметр `-b`). Подробнее про то, где в `vowpal wabbit` используется хэш-функция, можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction). Как меняется качество при изменении этих параметров? Верно ли, что при увеличении значений параметров `--passes` и `-b` качество всегда не убывает и почему?

In [14]:
# YOUR CODE HERE
#  Теперь запустим больше проходов по выборке и увеличим размер хеша.
! vw -d train --loss_function logistic --oaa 5 -f model --skips 3 --ngram 4 --quiet --passes 5 --cache_file cache_file -b 20
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.54994
accuracy = 0.97112


Добавление фичей в основном только ухудшает качество. Лишь добавление суффиксов небольшой длины увеличило его. Увеличение числапроходов и размера хеша уменьшает качество еще сильнее.

Теперь интерес представляет то, какие признаки оказались наиболее важными для модели. Для этого сначала переведем модель в читаемый формат.

In [15]:
# Вернем лучшую модель, которая была
! vw -d train --loss_function logistic --oaa 5 -f model --affix +2 --quiet
! vw -i model -t test -r pred --quiet

In [16]:
! vw -i model -t --invert_hash model.readable train --quiet
! head -n 30 model.readable

Version 8.4.0
Id 
Min label:-50
Max label:50
bits:18
lda:0
0 ngram:
0 skip:
options: --affix +2 --oaa 5
Checksum: 32408175
:0
Constant:142048:-1.88053
Constant[1]:142049:-2.94652
Constant[2]:142050:-2.42378
Constant[3]:142051:1.91574
Constant[4]:142052:-2.59536
a:216464:-0.0159857
a[1]:216465:0.129169
a[2]:216466:-0.0123258
a[3]:216467:0.00122789
a[4]:216468:0.0639045
aa:36168:-1.56558
aa[1]:36169:-1.64298
aa[2]:36170:-2.50951
aa[3]:36171:0.887328
aa[4]:36172:-1.23849
aaa:196024:-1.24977
aaa[1]:196025:-0.972496
aaa[2]:196026:-0.842369
aaa[3]:196027:0.945228


Первые несколько строк соответствуют информации о модели. Далее следуют строчки вида `feature[label]:hash:weight`. Выделите для каждого класса 10 признаков с наибольшими по модулю весами. Постарайтесь сделать ваш алгоритм прохода по файлу константным по памяти. Например, можно воспользоваться [кучей](https://docs.python.org/2/library/heapq.html).

In [17]:
# YOUR CODE HERE
import re
from heapq import heappop, heappush

with open('model.readable') as file:
    counter = 0
    label_regular = re.compile(r'\[\d\]')
    feature_regular = re.compile(r'^(.+):\d+:')
    weight_reqular = re.compile(r':\d+:(.+)$')
    heaps = {i: [] for i in STATUS_DICT.values()}
    heap_size = 10
    for line in file:
        if counter > 10:
            label = label_regular.findall(line)
            if len(label) != 0:
                label = int(label[0][1:-1])
            else:
                label = None
            feature = feature_regular.findall(line)[0]
            if feature[-1] == ']':
                feature = feature[:-3]
            weight = float(weight_reqular.findall(line)[0])

            if label is not None:
                if ((weight, feature) not in heaps[label]):
                    heappush(heaps[label], (weight, feature))
                    if len(heaps[label]) > heap_size:
                        heappop(heaps[label])
                else:
                    print('!!!')
        counter += 1

Посмотрим на кучи для некоторых статусов:

In [18]:
heaps[1]

[(1.46947, 'interview'),
 (1.46947, 'javajse'),
 (1.46947, 'petri'),
 (1.46947, 'xdebugfilelinkformat'),
 (1.46947, 'javalangruntime'),
 (1.46947, 'tun'),
 (1.78573, 'iphoneobjectivecc'),
 (1.78573, 'destroyed'),
 (1.78573, 'appendstring'),
 (1.78573, 'books')]

In [19]:
heaps[2]

[(1.05868, 'ubuntu'),
 (1.05868, 'usermetadata'),
 (1.09357, 'affix^+2=ub'),
 (1.09357, 'checknot'),
 (1.05868, 'webxm'),
 (1.09357, 'hashmapunorderd'),
 (1.09357, 'skbuffs'),
 (1.09357, 'sqlsrvfetchobject'),
 (1.09357, 'resolutiontiming'),
 (1.09357, 'integrityerror')]

In [20]:
heaps[4]

[(0.525523, 'sshconnent'),
 (0.525523, 'statsd'),
 (0.540231, 'remarks'),
 (0.540231, 'affix^+2=ch'),
 (0.525523, 'surfacegame'),
 (0.540231, 'splicelike'),
 (0.540231, 'servergo'),
 (0.540231, 'chackunchack'),
 (0.540231, 'popuphtml'),
 (0.540231, 'mapthread')]

Интересно, что кое-где суффиксы являются сильно значимыми признаками. Таким образом и следует выбирать, какие фичи добавлять.

Добавим признаки, извлеченные из текста поста (поле `BodyMarkdown`). В этом поле находится более подробная информация о вопросе, и часто туда помещают код, формулы и т.д. При удалении пунктуации мы потеряем много полезной информации, однако модель "мешка слов" на сырых данных может сильно раздуть признаковое пространство. В таких случаях работают с n-граммами на символах. <br>
Будьте осторожны: символы "`:`" и "`|`" нельзя использовать в названиях признаков, поскольку они являются служебными для `vw`-формата. Замените эти символы на два других редко встречающихся в выборке (или вообще не встречающихся). Также не забудьте про "`\n`". <br>
Поскольку для каждого документа одна n-грамма может встретиться далеко не один раз, то будет экономнее записывать признаки в формате `[n-грамма]:[число вхождений]`.

Также добавьте тэги (поля вид `TagN`). Приветствуется добавление информации о пользователе из других полей. Только не используйте `PostClosedDate` – в нем содержится информация о таргете.

In [21]:
from collections import Counter
import re


def extract_ngram_body(row, ngram=3):
    # YOUR CODE HERE
    body_text = re.sub('\|', 'ы', row['BodyMarkdown'])
    body_text = re.sub('\:', 'я', body_text)
    body_text = re.sub('\n+', '. ', body_text)
    def ngram_from_word(counter, word, ngram=2):
        for index in range(1, len(word) - ngram + 1):
            counter[word[index:(index + ngram)]] += 1
    ngrams_dict = Counter()
    for word in body_text.split(' '):
        ngram_from_word(ngrams_dict, word, ngram)
    return ' '.join([key + ':' + str(ngrams_dict[key]) for key in ngrams_dict])


def extract_tags(row):
    # YOUR CODE HERE
    return ' '.join(row['Tag' + str(i)] for i in range(1, 6))

Объединим все вместе. Реализуйте экстрактор признаков, который выделяет каждую подгруппу в отдельный namespace.

In [22]:
extractors_list = [
    ('t', extract_title),
    ('b', extract_ngram_body),
    ('a', extract_tags)
] # (namespace, extractor)


def make_feature_extractor(extractors_list):
    def feature_extractor(row):
        # YOUR CODE HERE
        return ' '.join(['|' + name + ' ' + extractor(row) for name, extractor in extractors_list])
    return feature_extractor

In [23]:
data2vw(make_feature_extractor(extractors_list))
! head -n 1 train

:1 dou:1 |a c#    


In [24]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14086
accuracy = 0.97858


Поэкспериментируйте с другими параметрами модели. Добавьте квадратичные взаимодействия между различными блоками признаков, измените параметры оптимизатора, добавьте регуляризацию и т.д. Совсем необязательно перебирать все параметры по сетке и добиваться оптимального качества. Для нас важнее то, насколько хорошо вы разобрались с возможностями библиотеки и продемонстрировали это.

Выберите не менее трех параметров. Для каждого из них объясните, почему по вашему мнению его изменение может улучшить качество модели, подберите оптимальное значение. Можете перебрать несколько значений "руками", а можете воспользоваться [vw-hypersearch](https://github.com/JohnLangford/vowpal_wabbit/wiki/Using-vw-hypersearch) или [vw-hyperopt](https://github.com/JohnLangford/vowpal_wabbit/blob/master/utl/vw-hyperopt.py) ([статья на хабре](https://habrahabr.ru/company/dca/blog/272697/)). Какие параметры повлияли на улучшение качества сильнее всего?

In [25]:
# YOUR CODE HERE

Добавим в фичи n-граммы разных длин.

In [26]:
# Добавим n-граммы разной длины
extractors_list = [
    ('t', extract_title),
    ('b', lambda row: extract_ngram_body(row, 2)),
    ('c', lambda row: extract_ngram_body(row, 3)),
    ('d', lambda row: extract_ngram_body(row, 4)),
    ('a', extract_tags)
] # (namespace, extractor)

data2vw(make_feature_extractor(extractors_list))

In [27]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14563
accuracy = 0.97764


Добавлене таких фичей существенно качество не улучшило. Но небольшой прирост все же дало.

Попробуем l1 и l2 регуляризацию

In [28]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l2 0.000001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.12954
accuracy = 0.97925


In [29]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l1 0.000001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.15668
accuracy = 0.97927


Как видим, l2 регуляризация дала значителное улучшение, а l1 оказалась бесполезной. Лучше брать l2 регуляризацию.

Теперь попробуем улучшить качество, поменяв различный параметры оптимизации.

In [30]:
! vw -d train --loss_function logistic --oaa 5 -f model --sgd --quiet --passes 1 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 12.72904
accuracy = 0.50755


In [31]:
! vw -d train --loss_function logistic --oaa 5 -f model --sgd --quiet --passes 5 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 11.29637
accuracy = 0.53042


In [32]:
! vw -d train --loss_function logistic --oaa 5 -f model --sgd --quiet --passes 10 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 11.92278
accuracy = 0.48418


Интересно, что для sgd увеличение числа проходов все же положительно складывается на качестве.

In [33]:
! vw -d train --loss_function logistic --oaa 5 -f model --adaptive --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 2.00079
accuracy = 0.95444


Однако в целом изменение режима оптимизации приводит к значительно худшим результатам.

Попробуем уменьшаять learning rate между проходами.

In [37]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet --l2 0.000001 --passes 3 --cache_file cache_file --decay_learning_rate 0.05
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores()) 

logloss  = 0.43511
accuracy = 0.94437


Прироста достигнуть не получилось.

В целом все ясно, самая главная команда: vw -h